In [1]:
from rdflib import Graph

import sys
sys.path.insert(0,'..')

from write_data import *

### The goal of this notebook is to retrieve the data from OWL

In [2]:
g = Graph()
g.parse("owl.ttl", encoding="utf-8")
context = "http://www.w3.org/2002/07/owl#"

### Retrieve the properties

In [3]:
q = """
    SELECT distinct ?prop ?type
    WHERE {
      
      VALUES ?type { <http://www.w3.org/1999/02/22-rdf-syntax-ns#Property> 
                     <http://www.w3.org/2002/07/owl#DatatypeProperty>
                     <http://www.w3.org/2002/07/owl#ObjectProperty> }
      
      ?prop <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> ?type.
    }
    """

dict_prop = {}
for r in g.query(q):
    if r["prop"].n3() in dict_prop:
        dict_prop[r["prop"].n3()]["type"].add(r["type"].n3())
    else:
        dict_prop[r["prop"].n3()] = {"type":{r["type"].n3()}, "context":context, "label":set(), "comment":set(), "domain":set(), "range":set()}

In [4]:
q = """
    SELECT distinct ?prop ?label
    WHERE {
      ?prop <http://www.w3.org/2000/01/rdf-schema#label> ?label.
      
      {?prop <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/1999/02/22-rdf-syntax-ns#Property>}
      UNION
      {?prop <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#DatatypeProperty>}
      UNION
      {?prop <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#ObjectProperty>}
    }
    """

for r in g.query(q):
    dict_prop[r["prop"].n3()]["label"].add(r["label"].n3())    

In [5]:
q = """
    SELECT distinct ?prop ?comment
    WHERE {
      
      ?prop <http://www.w3.org/2000/01/rdf-schema#comment> ?comment.
      
      {?prop <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/1999/02/22-rdf-syntax-ns#Property>}
      UNION
      {?prop <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#DatatypeProperty>}
      UNION
      {?prop <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#ObjectProperty>}

    }
    """
for r in g.query(q):
    dict_prop[r["prop"].n3()]["comment"].add(r["comment"].n3())

In [6]:
q = """
    SELECT distinct ?prop ?label ?domain
    WHERE {
      
      ?prop <http://www.w3.org/2000/01/rdf-schema#domain> ?domain.
      
      {?prop <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/1999/02/22-rdf-syntax-ns#Property>}
      UNION
      {?prop <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#DatatypeProperty>}
      UNION
      {?prop <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#ObjectProperty>}
      
    }
    """

for r in g.query(q):
    
    dict_prop[r["prop"].n3()]["domain"].add(str(r["domain"]))

In [7]:
q = """
    SELECT distinct ?prop ?range
    WHERE {
      
      ?prop <http://www.w3.org/2000/01/rdf-schema#range> ?range.
      
      {?prop <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/1999/02/22-rdf-syntax-ns#Property>}
      UNION
      {?prop <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#DatatypeProperty>}
      UNION
      {?prop <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#ObjectProperty>}
      
    }
    """

for r in g.query(q):
    
    dict_prop[r["prop"].n3()]["range"].add(str(r["range"]))

In [8]:
f = open("Properties.nt", "w", encoding="utf-8")

write_property(f, dict_prop)
    
f.close()

### Retrieve the Classes

In [9]:
q = """
    SELECT distinct ?class ?label
    WHERE {
      ?class <http://www.w3.org/2000/01/rdf-schema#label> ?label.
      
      ?class <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Class>.
    }
    """

dict_classes = {}

for r in g.query(q):
    
    dict_classes[r["class"].n3()] = {"label":{r["label"].n3()}, "comment":set(), "context":context}

In [10]:
q = """
    SELECT distinct ?class  ?comment
    WHERE {
      
      ?class <http://www.w3.org/2000/01/rdf-schema#comment> ?comment.
      
      ?class <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Class>.
      
    }
    """

for r in g.query(q):
    
    dict_classes[r["class"].n3()]["comment"].add(r["comment"].n3())

In [11]:
f = open("Classes.nt", "w", encoding="utf-8")

write_class(f, dict_classes)
    
f.close()